### Import Libraries & Modules

In [ ]:
# Auto reload modules -> helps with development
%load_ext autoreload
%autoreload 2

# Import Libraries
import os
from pathlib import Path

# Import Modules
from data.data_puller import DataPuller
from config import load_config
from utils.logging_config import setup_logging
from data.data_validator import DataValidator
from engine.sim_engine import SimEngine


### Load Configs

In [ ]:
# Configuration settings
config_file = 'pick_optimization/config/config.yaml'

# Load configuration
main_dir = Path.cwd().parent 
config_path = os.path.join(main_dir, config_file)
config = load_config(config_path)

# Set up logging
logger = setup_logging(config)

    
logger.info("Starting Pick Planning Model execution")

# Get input mode and parameters
input_mode = config['global']['input_mode']
fc = config['global']['wh_id']
start_time = config['global']['start_time']
end_time = config['global']['end_time']


### Load Input Data

In [ ]:
# Initialize data puller
data_puller = DataPuller(config_path, input_mode)

# Pull required data
logger.info("Fetching input data...")
container_data = data_puller.get_container_data(fc, start_time, end_time)
slotbook_data = data_puller.get_slotbook_data(fc)

# Validate input data
validator = DataValidator(config)
container_data, slotbook_data = validator.validate(container_data, slotbook_data)


### Run Pick Optimization Model

In [ ]:
# Initialize and run simulation engine
logger.info("Initializing simulation engine...")
engine = SimEngine(config)

results = engine.run(
    container_data=container_data,
    slotbook_data=slotbook_data,
    start_time=start_time,
    end_time=end_time
)
